In [1]:
import numpy as np

from bokeh.models import ColumnDataSource, DataRange1d
from bokeh.models.glyphs import AnnularWedge, Ellipse, Quad
from bokeh.io import curdoc, show
from bokeh.plotting import figure
import queue


open_space_top = 0
open_space_bottom = 0
open_space_left = 0
open_space_right = 0

In [2]:
def Foamtree(node):
    if node.children:
        global open_space_left 
        global open_space_top
        global open_space_bottom
        global open_space_right
        on_width = False
        for n in node.children:
            if not n.seen:
                n.area = (n.leaves_subtree/ n.parent.leaves_subtree) * n.parent.area
                if (np.roots(open_space_right**2 - open_space_left**2)) >= (np.roots(open_space_top**2 - open_space_bottom**2)):
                    n.top = open_space_top
                    n.bottom = open_space_bottom
                    n.left = open_space_left
                    n.right = (n.area / (n.top - n.bottom)) + n.left
                    on_width = True
                else:
                    n.top = open_space_top
                    n.left = open_space_left
                    n.right = open_space_right
                    n.bottom = (n.area / (n.right - n.left)) + n.top
                    on_width = False
                glyph = Quad(left=n.left, right=n.right, top=n.top, bottom=n.bottom, fill_color=n.color, fill_alpha = 0.5)
                plot.add_glyph(source, glyph)
                if on_width:
                    open_space_left = n.right
                else: 
                    open_space_top = n.bottom
                Foamtree(n)
            n.seen = True

In [3]:
def Foamroot(root):
    global open_space_left 
    global open_space_top
    global open_space_bottom
    global open_space_right
    root.top = 14
    root.bottom = 6
    root.left = 6
    root.right = 14
    open_space_top = root.top
    open_space_bottom = root.bottom
    open_space_left = root.left
    open_space_right = root.right
    glyph = Quad(left=root.left, right=root.right, top=root.top, bottom=root.bottom, fill_color=root.color, fill_alpha = 0.5)
    plot.add_glyph(source, glyph)
    Foamtree(root)

In [4]:
class Node:
    def __init__(self, level, children, leaves_subtree, color):
        self.color = color
        self.level = level
        self.children = children
        self.leaves_subtree = leaves_subtree
        self.width = 1
        self.range = 1
        self.right_bound = 0
        self.left_bound = 0
        self.parent = None
        self.seen = False
        self.top = 0
        self.bottom = 0
        self.right = 0
        self.left = 0
        self.area = 0

In [5]:
def Set_parent(root):
    if root.children:
        for n in root.children:
            n.parent = root
            Set_parent(n)

In [6]:
def Reset(root):
    if root.children:
        for n in root.children:
            n.seen = False
            Reset(n)

In [7]:
m = Node(3, None, 1, '#CDE6FF')
n = Node(3, None, 1, '#860000')
l = Node(2, [m], 1,'#75FFE7') 
f = Node(2, None, 1, '#191970') 
g = Node(2, None, 1, '#CD1076') 
h = Node(2, [n], 1, '#696969')
i = Node(2, None, 1, '#83563C')
j = Node(2, None, 1, '#F0F8FF')
k = Node(2, None, 1, '#7B322C')
e = Node(1, [l,k,j], 3, '#7F1AC4')
b = Node(1, [f], 1, '#FFFF66')
c = Node(1, [g, h], 2, '#FF758D')
d = Node(1, [i], 1, '#81E9EF')
a = Node(0, [b,c,d,e], 7, '#FF0000') 

In [8]:
Set_parent(a)

In [9]:
source = ColumnDataSource()

xdr = DataRange1d(start=5, end=15)
ydr = DataRange1d(start=5, end=15)

plot = figure(title=None, x_range=xdr, y_range=ydr, plot_width=320, plot_height=300,
              h_symmetry=False, v_symmetry=False, min_border=0, 
              tools="pan,wheel_zoom,box_zoom,reset,save,lasso_select,box_select,hover")


Foamroot(a)

curdoc().add_root(plot)

show(plot)
Reset(a)